In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
from scipy.stats import skew, norm

dff=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
dff

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [2]:
for df in [dff,df_test]:
    df.drop('Id',axis=1,inplace=True)


In [4]:
for train_test in [dff,df_test]:
# Converting non-numeric predictors stored as numbers into string

    train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
    train_test['YrSold'] = train_test['YrSold'].apply(str)
    train_test['MoSold'] = train_test['MoSold'].apply(str)

    # Filling Categorical NaN (That we know how to fill due to the description file )

    train_test['Functional'] = train_test['Functional'].fillna('Typ')
    train_test['Electrical'] = train_test['Electrical'].fillna("SBrkr")
    train_test['KitchenQual'] = train_test['KitchenQual'].fillna("TA")
    train_test['Exterior1st'] = train_test['Exterior1st'].fillna(train_test['Exterior1st'].mode()[0])
    train_test['Exterior2nd'] = train_test['Exterior2nd'].fillna(train_test['Exterior2nd'].mode()[0])
    train_test['SaleType'] = train_test['SaleType'].fillna(train_test['SaleType'].mode()[0])
    train_test["PoolQC"] = train_test["PoolQC"].fillna("None")
    train_test["Alley"] = train_test["Alley"].fillna("None")
    train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna("None")
    train_test['Fence'] = train_test['Fence'].fillna("None")
    train_test['MiscFeature'] = train_test['MiscFeature'].fillna("None")

    for col in ('GarageArea', 'GarageCars'):
        train_test[col] = train_test[col].fillna(0)
            
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        train_test[col] = train_test[col].fillna('None')
        
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        train_test[col] = train_test[col].fillna('None')
        
        # Checking the features with NaN remained out
    train_test["SqFtPerRoom"] = train_test["GrLivArea"] / (train_test["TotRmsAbvGrd"] +
                                                       train_test["FullBath"] +
                                                       train_test["HalfBath"] +
                                                       train_test["KitchenAbvGr"])

    train_test['Total_Home_Quality'] = train_test['OverallQual'] + train_test['OverallCond']

    train_test['Total_Bathrooms'] = (train_test['FullBath'] + (0.5 * train_test['HalfBath']) +
                                train_test['BsmtFullBath'] + (0.5 * train_test['BsmtHalfBath']))

    train_test["HighQualSF"] = train_test["GrLivArea"]+train_test["1stFlrSF"] + train_test["2ndFlrSF"]+0.5*train_test["GarageArea"]+0.5*train_test["TotalBsmtSF"]+1*train_test["MasVnrArea"]

    train_test["Age"] = pd.to_numeric(train_test["YrSold"])-pd.to_numeric(train_test["YearBuilt"])


    for col in train_test:
        if train_test[col].isna().sum() > 0:
            print(train_test[col][0])

65.0
BrkFace
196.0
2003.0
4318.0
RH
80.0
AllPub
None
0.0
468.0
144.0
270.0
882.0
0.0
0.0
1961.0
1.0
2598.0


In [5]:
for df in [dff,df_test]:
    df.drop(['PoolQC','MiscFeature','Alley','Fence','YearRemodAdd','FireplaceQu'],axis=1,inplace=True)

In [6]:
for df in [dff,df_test]:
    for i in df.columns:
        if df[i].dtypes=='object':
            df[i]=df[i].astype('category').cat.codes
dff


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,Age
0,9,3,65.0,8450,1,3,3,0,4,0,...,4,2,8,4,208500,142.500000,12,3.5,4318.0,5
1,4,3,80.0,9600,1,3,3,0,2,0,...,7,1,8,4,181500,140.222222,14,2.5,3385.0,31
2,9,3,68.0,11250,1,0,3,0,4,0,...,11,2,8,4,223500,178.600000,12,3.5,4498.0,7
3,10,3,60.0,9550,1,0,3,0,0,0,...,4,0,8,0,140000,190.777778,12,2.0,4133.0,91
4,9,3,84.0,14260,1,0,3,0,2,0,...,3,2,8,4,250000,169.076923,13,3.5,5736.5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,9,3,62.0,7917,1,3,3,0,4,0,...,10,1,8,4,175000,149.727273,11,2.5,4000.5,8
1456,4,3,85.0,13175,1,3,3,0,4,0,...,4,4,8,4,210000,207.300000,12,3.0,5286.0,32
1457,10,3,66.0,9042,1,3,3,0,4,0,...,7,4,8,4,266500,195.000000,16,2.0,5382.0,69
1458,4,3,68.0,9717,1,3,3,0,4,0,...,6,4,8,4,142125,154.000000,11,2.0,2815.0,60


In [7]:
total = df_test.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
LotFrontage,227,0.155586
GarageYrBlt,78,0.053461
HighQualSF,16,0.010966
MasVnrArea,15,0.010281
Total_Bathrooms,2,0.001371
BsmtFullBath,2,0.001371
BsmtHalfBath,2,0.001371
BsmtUnfSF,1,0.000685
TotalBsmtSF,1,0.000685
BsmtFinSF2,1,0.000685


In [8]:
for df in [dff,df_test]:
    df['GarageYrBlt'].fillna(df['GarageYrBlt'].mode()[0],inplace=True)

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
IMPUTE = True

In [10]:
def analyze_column(input_series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(input_series):
        return 'numeric'
    else:
        return 'categorical'
    


class LGBMImputer:
    '''
    Regression imputer using LightGBM
    '''
    def __init__(self, cat_features=[], n_iter=15000, verbose=False):
        self.n_iter = n_iter
        self.cat_features = cat_features
        self.verbose = verbose
        self.n_features = None
        self.feature_names = None
        self.feature_with_missing = None
        self.imputers = {}
        self.offsets = {}
        self.objectives = {} 
                                                                            # Özellikler-Parametreler belirleniyor...
        
    def fit_transform(self, X, y=None):
        output_X = X.copy()
        self.n_features = X.shape[1]

        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
        else:
            self.feature_names = [f'f{i}' for i in range(self.n_features)]
            X = pd.DataFrame(X, columns=self.feature_names)
        self.feature_with_missing = [col for col in self.feature_names if X[col].isnull().sum() > 0]

        for icol, col in enumerate(self.feature_with_missing):              #icol sıra numarası, col kolon
            if icol in self.cat_features:
                nuni = X[col].dropna().nunique()
                if nuni == 2:
                    params = {
                        'objective': 'binary'
                    }
                elif nuni > 2:
                    params = {
                        'objective': 'multiclass',
                        'num_class': nuni + 1
                    }
            else: # automatic analyze column
                if analyze_column(X[col]) == 'numeric':
                    params = {
                        'objective': 'regression'
                    }
                else:
                    nuni = X[col].dropna().nunique()
                    if nuni == 2:
                        params = {
                            'objective': 'binary'
                        }
                    elif nuni > 2:
                        params = {
                            'objective': 'multiclass',
                            'num_class': nuni + 1
                        }
                    else:
                        print(f'column {col} has only one unique value.')
                        continue

            params['verbosity'] = -1
            
            null_idx = X[col].isnull()
            x_train = X.loc[~null_idx].drop(col, axis=1)
            x_test = X.loc[null_idx].drop(col, axis=1)
            y_offset = X[col].min()
            y_train = X.loc[~null_idx, col].astype(int) - y_offset
            dtrain = lgb.Dataset(
                data=x_train,
                label=y_train
            )

            early_stopping_rounds = 50
            model = lgb.train(
                params, dtrain, valid_sets=[dtrain], 
                num_boost_round=self.n_iter,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=0,
            )

            y_test = model.predict(x_test)
            if params['objective'] == 'multiclass':
                y_test = np.argmax(y_test, axis=1).astype(float)
            elif params['objective'] == 'binary':
                y_test = (y_test > 0.5).astype(float)
            y_test += y_offset
            output_X.loc[null_idx, col] = y_test
            if params['objective'] in ['multiclass', 'binary']:
                output_X[col] = output_X[col].astype(int)
            self.imputers[col] = model
            self.offsets[col] = y_offset
            self.objectives[col] = params['objective']
            if self.verbose:
                print(f'{col}:\t{self.objectives[col]}...iter{model.best_iteration}/{self.n_iter}')
        
        return output_X    

In [11]:
if IMPUTE:
    s=0
    for df in [dff,df_test]:
        feature_df = df
        feature_df = feature_df.replace(99999.0,np.nan)

        imputer = LGBMImputer(verbose=True)
        feature_df.loc[:, feature_df.columns[1:-1]] = imputer.fit_transform(feature_df[feature_df.columns[1:-1]])

        if s==0:
            train_df = feature_df.copy().reset_index(drop=True)
            s=1
        else:
            test_df = feature_df.copy().reset_index(drop=True)

c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


LotFrontage:	regression...iter14985/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


MasVnrArea:	regression...iter15000/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

BsmtHalfBath:	regression...iter15000/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Total_Bathrooms:	regression...iter14460/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


HighQualSF:	regression...iter15000/15000


In [12]:
train_df.isna().sum().sum()

0

In [13]:
train_df.to_csv('trainimp.csv')
test_df.to_csv('testimp.csv')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import seaborn as sbn

df=pd.read_csv('trainimp.csv')
df_test=pd.read_csv('testimp.csv')

In [2]:
X_train = df.drop("SalePrice", axis=1)
y_train = df.SalePrice

X_test = df_test.copy()

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from catboost import CatBoostRegressor


In [ ]:
regressor = {
    'GB':GradientBoostingRegressor(),
    "SGD": SGDRegressor(),
    "KNN": KNeighborsRegressor(), 
    "LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeRegressor(),
    "RF": RandomForestRegressor(max_depth=3, random_state=2),
    "SVM": SVC(),
    "MLP": MLPRegressor(max_iter=1000),
    "XGB": XGBRegressor(),
    "LGBM": LGBMRegressor(),
    'CB': CatBoostRegressor(),
}

results = pd.DataFrame(columns=["Regressor", "rmse", "r2"])
for name, clf in regressor.items():
    model = clf
    clf.fit(X_train_scaled,y_train)
    predictions = clf.predict(X_train_scaled)
    results = results.append({
        "Regressor": name,
        "rmse": np.sqrt(np.square(y_train - predictions)).mean(),
        "r2": 1 - np.square(y_train - predictions).sum()/np.square(y_train - y_train.mean()).sum(),
    }, ignore_index=True)
    
results



In [ ]:
clf=SGDRegressor()
#clf.fit(X_train_scaled,y_train)

params = {
    'penalty':('l2', 'l1', 'elasticnet'),
    'alpha':(0.0001,0.001,0.00001,0.01),
    'max_iter':(1000,1200,1500,1300),
    'tol': (0.0001,0.001,0.00001,0.01),
}
clf = GridSearchCV(clf, params, cv=10)
clf.fit(X_train_scaled, y_train)
print("Best hyperparameter:", clf.best_params_)

#Score: 6.63719


In [5]:
svm = GradientBoostingRegressor()
params = {
    'n_estimators' :(50,75),
    'learning_rate' : (0.08,0.01)
}
clf = GridSearchCV(svm, params, cv=10)
clf.fit(X_train_scaled, y_train)
print("Best hyperparameter:", clf.best_params_)

Best hyperparameter: {'learning_rate': 0.08, 'n_estimators': 75}


In [13]:
clf=CatBoostRegressor()
clf.fit(X_train_scaled,y_train)

Learning rate set to 0.043466
0:	learn: 77325.5720031	total: 2.44ms	remaining: 2.43s


SystemError: <method '_train' of '_catboost._CatBoost' objects> returned a result with an exception set

In [19]:
y_pred = clf.predict(X_test_scaled)
submission = pd.read_csv("sample_submission.csv")
submission["SalePrice"] = y_pred[:1459]
submission.to_csv('submission.csv', index=False)

In [20]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m ""

Successfully submitted to House Prices - Advanced Regression Techniques



  0%|          | 0.00/35.1k [00:00<?, ?B/s]
 23%|██▎       | 8.00k/35.1k [00:00<00:00, 74.7kB/s]
100%|██████████| 35.1k/35.1k [00:01<00:00, 18.4kB/s]
